## feature eng

### 1차 가공

In [529]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import urllib.request
import urllib.request
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tqdm import tqdm
import os
from imblearn.over_sampling import SMOTE

print('-------------------')
print('|     lab1         |')
print('-------------------')

if os.path.exists('/semi.csv'):
    '''
    혹시 파일이 생기지 않는다면, 아래 두 줄의 스크립스틑 파이썬 .py파일로 만들어서 실행하면 됩니다. 
    '''
    url = "https://drive.google.com/uc?export=download&id=1XCU0eo2xZ03xhxJhdrCnVjduCoaBQ7kJ"
    urllib.request.urlretrieve(url, "semi.csv")  # save in a file
else:
    print('data already exist')


-------------------
|     lab1         |
-------------------
data already exist


In [530]:
df = pd.read_csv('semi.csv')

In [531]:
df.isnull().sum().sum()


np.int64(41951)

In [532]:
df.isna().sum().sum()

np.int64(41951)

In [533]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Columns: 592 entries, Time to Pass/Fail
dtypes: float64(590), int64(1), object(1)
memory usage: 7.1+ MB


In [534]:
df.describe()


,0,1,2,3,4,5,6,7,8,9,...,581,582,583,584,585,586,587,588,589,Pass/Fail
count,1561.000000,1560.000000,1553.000000,1553.000000,1553.000000,1553.0,1553.000000,1558.000000,1565.000000,1565.000000,...,618.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1567.000000
mean,3014.452896,2495.850231,2200.547318,1396.376627,4.197013,100.0,101.112908,0.121822,1.462862,-0.000841,...,97.934373,0.500096,0.015318,0.003847,3.067826,0.021458,0.016475,0.005283,99.670066,-0.867262
std,73.621787,80.407705,29.513152,441.691640,56.355540,0.0,6.237214,0.008961,0.073897,0.015116,...,87.520966,0.003404,0.017180,0.003720,3.578033,0.012358,0.008808,0.002867,93.891919,0.498010
min,2743.240000,2158.750000,2060.660000,0.000000,0.681500,100.0,82.131100,0.000000,1.191000,-0.053400,...,0.000000,0.477800,0.006000,0.001700,1.197500,-0.016900,0.003200,0.001000,0.000000,-1.000000
25%,2966.260000,2452.247500,2181.044400,1081.875800,1.017700,100.0,97.920000,0.121100,1.411200,-0.010800,...,46.184900,0.497900,0.011600,0.003100,2.306500,0.013425,0.010600,0.003300,44.368600,-1.000000
50%,3011.490000,2499.405000,2201.066700,1285.214400,1.316800,100.0,101.512200,0.122400,1.461600,-0.001300,...,72.288900,0.500200,0.013800,0.003600,2.757650,0.020500,0.014800,0.004600,71.900500,-1.000000
75%,3056.650000,2538.822500,2218.055500,1591.223500,1.525700,100.0,104.586700,0.123800,1.516900,0.008400,...,116.539150,0.502375,0.016500,0.004100,3.295175,0.027600,0.020300,0.006400,114.749700,-1.000000
max,3356.350000,2846.440000,2315.266700,3715.041700,1114.536600,100.0,129.252200,0.128600,1.656400,0.074900,...,737.304800,0.509800,0.476600,0.104500,99.303200,0.102800,0.079900,0.028600,737.304800,1.000000


In [535]:
df = df.dropna(thresh=1)  # 최소 2개의 유효 값이 있어야 삭제되지 않음


In [536]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,581,582,583,584,585,586,587,588,589,Pass/Fail
count,1561.000000,1560.000000,1553.000000,1553.000000,1553.000000,1553.0,1553.000000,1558.000000,1565.000000,1565.000000,...,618.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1567.000000
mean,3014.452896,2495.850231,2200.547318,1396.376627,4.197013,100.0,101.112908,0.121822,1.462862,-0.000841,...,97.934373,0.500096,0.015318,0.003847,3.067826,0.021458,0.016475,0.005283,99.670066,-0.867262
std,73.621787,80.407705,29.513152,441.691640,56.355540,0.0,6.237214,0.008961,0.073897,0.015116,...,87.520966,0.003404,0.017180,0.003720,3.578033,0.012358,0.008808,0.002867,93.891919,0.498010
min,2743.240000,2158.750000,2060.660000,0.000000,0.681500,100.0,82.131100,0.000000,1.191000,-0.053400,...,0.000000,0.477800,0.006000,0.001700,1.197500,-0.016900,0.003200,0.001000,0.000000,-1.000000
25%,2966.260000,2452.247500,2181.044400,1081.875800,1.017700,100.0,97.920000,0.121100,1.411200,-0.010800,...,46.184900,0.497900,0.011600,0.003100,2.306500,0.013425,0.010600,0.003300,44.368600,-1.000000
50%,3011.490000,2499.405000,2201.066700,1285.214400,1.316800,100.0,101.512200,0.122400,1.461600,-0.001300,...,72.288900,0.500200,0.013800,0.003600,2.757650,0.020500,0.014800,0.004600,71.900500,-1.000000
75%,3056.650000,2538.822500,2218.055500,1591.223500,1.525700,100.0,104.586700,0.123800,1.516900,0.008400,...,116.539150,0.502375,0.016500,0.004100,3.295175,0.027600,0.020300,0.006400,114.749700,-1.000000
max,3356.350000,2846.440000,2315.266700,3715.041700,1114.536600,100.0,129.252200,0.128600,1.656400,0.074900,...,737.304800,0.509800,0.476600,0.104500,99.303200,0.102800,0.079900,0.028600,737.304800,1.000000


In [537]:
df.isna().sum().sum()

np.int64(41951)

In [538]:
def DuplicatedHandler(df: pd.DataFrame):
    features = range(590)
    stack = []
    idx = []
    for i, fe in enumerate(features):
        st = df[str(fe)].std()
        me = df[str(fe)].mean()
        mx = df[str(fe)].max()
        mn = df[str(fe)].mean()
        if (st, me, mx, mn) in stack:
            idx.append(fe)
            print('duplicated!', i)
        else:
            stack.append((st, me, mx, mn))

    for i in idx:
        df = df.drop(str(i), axis=1)
    df = df.drop(['Time'], axis=1)
    return df


df = DuplicatedHandler(df)
feature = df.drop('Pass/Fail', axis=1)
target = df['Pass/Fail']
X_train, X_test, y_train, y_test = train_test_split(feature, target, \
                                                    test_size=0.2, random_state=11, stratify=target)

X_train, X_test, y_train, y_test = pd.DataFrame(X_train), pd.DataFrame(X_test), pd.DataFrame(y_train), pd.DataFrame(
    y_test)

duplicated! 52
duplicated! 69
duplicated! 97
duplicated! 141
duplicated! 149
duplicated! 178
duplicated! 179
duplicated! 186
duplicated! 189
duplicated! 190
duplicated! 191
duplicated! 192
duplicated! 193
duplicated! 194
duplicated! 226
duplicated! 229
duplicated! 230
duplicated! 231
duplicated! 232
duplicated! 233
duplicated! 234
duplicated! 235
duplicated! 236
duplicated! 237
duplicated! 240
duplicated! 241
duplicated! 242
duplicated! 243
duplicated! 256
duplicated! 257
duplicated! 258
duplicated! 259
duplicated! 260
duplicated! 261
duplicated! 262
duplicated! 263
duplicated! 264
duplicated! 265
duplicated! 266
duplicated! 276
duplicated! 284
duplicated! 313
duplicated! 314
duplicated! 315
duplicated! 322
duplicated! 325
duplicated! 326
duplicated! 327
duplicated! 328
duplicated! 329
duplicated! 330
duplicated! 364
duplicated! 369
duplicated! 370
duplicated! 371
duplicated! 372
duplicated! 373
duplicated! 374
duplicated! 375
duplicated! 378
duplicated! 379
duplicated! 380
duplicated!

In [539]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,580,581,582,583,584,585,586,587,588,589
425,3079.17,2405.56,2217.3777,1425.1041,1.7585,100.0,106.2556,0.1200,1.4794,-0.0198,...,0.0024,31.3771,0.5080,0.0139,0.0039,2.7328,0.0234,0.0073,0.0024,31.3771
412,2989.85,2501.88,2197.2333,1435.1460,0.9740,100.0,104.8733,0.1226,1.5330,-0.0059,...,NaN,NaN,0.4976,0.0184,0.0038,3.6932,0.0145,0.0213,0.0074,146.5131
115,3002.85,2502.05,2232.5889,1717.2750,1.6700,100.0,104.1067,0.1223,1.4518,0.0066,...,0.0042,45.6835,0.4995,0.0093,0.0026,1.8612,0.0364,0.0166,0.0042,45.6835
887,3007.75,2535.14,2216.5000,1111.5436,0.8373,100.0,99.9867,0.1205,1.4503,0.0149,...,NaN,NaN,0.4948,0.0099,0.0032,1.9956,0.0133,0.0139,0.0038,104.3034
328,2894.04,2490.06,2207.0444,1330.6718,1.3076,100.0,101.6778,0.1216,1.5546,-0.0107,...,NaN,NaN,0.4963,0.0208,0.0050,4.1959,0.0226,0.0106,0.0034,46.9253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,3045.48,2408.85,2223.0444,1194.5986,1.2016,100.0,112.5811,0.1229,1.3838,-0.0125,...,NaN,NaN,0.4990,0.0137,0.0032,2.7392,0.0230,0.0210,0.0053,91.4264
1447,2949.12,2553.24,2176.8000,1461.4374,0.8864,100.0,96.2367,0.1232,1.5576,-0.0205,...,NaN,NaN,0.5031,0.0111,0.0033,2.2148,0.0250,0.0088,0.0029,35.1564
1412,3025.46,2516.06,2195.9778,1388.2869,1.5605,100.0,103.2400,0.1234,1.4298,0.0122,...,0.0195,171.9936,0.4985,0.0169,0.0047,3.3884,0.0378,0.0651,0.0195,171.9936
270,2988.52,2291.92,2183.5777,1764.5386,1.7050,100.0,100.4478,0.1222,1.4305,0.0001,...,0.0046,51.9067,0.4999,0.0095,0.0028,1.9072,0.0274,0.0142,0.0046,51.9067


### Create feature Handler

In [540]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

def NullHandler(train, test):
    for column in train.columns:
        if train[column].isnull().sum() > 0:
            if train[column].dtype in ['int64', 'float64']:
                median_value = train[column].median()
                train[column] = train[column].fillna(median_value)
                test[column] = test[column].fillna(median_value)
            else:
                mode_value = train[column].mode()[0]
                train[column] = train[column].fillna(mode_value)
                test[column] = test[column].fillna(mode_value)
    return train, test

def OutlierHandler(train):
    for column in train.columns:
        if train[column].dtype in ['int64', 'float64']:
            Q1 = train[column].quantile(0.25)
            Q3 = train[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            # Replace outliers with NaN
            train[column] = np.where((train[column] < lower_bound) | (train[column] > upper_bound), np.nan, train[column])
    
    # Fill NaN values with median or mode
    train = train.apply(
        lambda col: col.fillna(col.median()) if col.dtype in ['int64', 'float64'] else col.fillna(col.mode()[0])
    )
    return train

def FeatureSelection(train, y_train):
    # 상관관계 기반 특성 제거
    corr_matrix = train.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))
    to_drop = [column for column in upper.columns if any(upper[column] > 0.98)]  # 임계값을 0.95에서 0.98로 조정
    train = train.drop(to_drop, axis=1)

    # 랜덤 포레스트 모델을 사용하여 중요도 기반 특성 선택
    model = RandomForestClassifier(n_estimators=100, random_state=1234)
    model.fit(train, y_train)
    feature_importances = pd.Series(model.feature_importances_, index=train.columns)
    
    # 임계값을 0.01에서 0.001로 조정하여 더 많은 특성 선택
    important_features = feature_importances[feature_importances > 0.001].index
    train = train[important_features]
    
    return train, important_features

def LogTransform(train, test):
    numeric_features = train.select_dtypes(include=['int64', 'float64']).columns
    # Use np.where to handle negative values before applying log
    train[numeric_features] = train[numeric_features].apply(lambda x: np.log1p(np.where(x < 0, 0, x)))
    test[numeric_features] = test[numeric_features].apply(lambda x: np.log1p(np.where(x < 0, 0, x)))
    return train, test

def DataHandler(X_train, X_test, y_train, y_test):
    X_train, X_test = NullHandler(X_train.copy(), X_test.copy())
    X_train = OutlierHandler(X_train)
    X_train, important_features = FeatureSelection(X_train, np.array(y_train).ravel())
    X_test = X_test[important_features]
    X_train, X_test = LogTransform(X_train, X_test)
    
    # Apply StandardScaler
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test, np.array(y_train).ravel(), np.array(y_test).ravel()

# Usage
X_train, X_test, y_train, y_test = DataHandler(X_train, X_test, y_train, y_test)

X_train, X_test, y_train, y_test = pd.DataFrame(X_train), pd.DataFrame(X_test), pd.DataFrame(y_train), pd.DataFrame(
    y_test)

In [541]:
print('is NaN data check : ', X_train.isna().sum().sum())
print('is NaN data check : ', X_test.isna().sum().sum())

is NaN data check :  0
is NaN data check :  27


In [542]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,337,338,339,340,341,342,343,344,345,346
0,1.100173,-1.547911,0.366238,1.345744,1.038217,-1.334828,0.218068,-0.616935,-0.708639,-1.196767,...,0.407950,-0.870255,-1.279422,2.475438,0.032709,0.452980,0.284561,-1.227876,-1.188515,-0.800278
1,-0.296798,0.105905,0.395700,-0.993616,0.740561,0.075047,0.935004,-0.616935,3.939245,-1.689306,...,1.580731,-0.706838,-1.538747,-0.756875,1.467442,0.310389,-0.599418,0.836866,1.169626,1.033285
2,-0.090903,0.108767,1.148780,1.117778,0.573807,-0.087464,-0.157169,0.241133,2.381254,0.154604,...,1.829441,0.127405,-0.153694,-0.164684,-1.440504,-1.401812,1.562047,0.146875,-0.338232,-0.357820
3,-0.013527,0.662231,-0.676303,-1.495323,-0.343764,-1.063444,-0.177683,1.312268,-0.502572,-0.705361,...,1.236275,-0.278016,-0.456263,-1.630949,-1.247966,-0.545456,-0.719199,-0.250946,-0.527052,0.625731
4,-1.842429,-0.093586,0.078576,0.097999,0.037313,-0.466826,1.219639,-0.616935,0.770465,-0.487321,...,0.297615,0.066462,0.881043,-1.162492,2.230043,2.020544,0.205417,-0.738614,-0.715947,-0.326077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248,0.578076,-1.490338,-0.374001,-0.230743,2.353112,0.237515,-1.099996,-0.616935,0.790934,0.881237,...,-0.464607,-0.466771,-0.084042,-0.320451,-0.031205,-0.545456,0.244996,0.792919,0.180635,0.468024
1249,-0.947725,0.961924,0.471871,-1.310949,-1.212077,0.399939,1.258981,-0.616935,1.240729,1.432693,...,0.344280,-0.182041,-0.204320,0.955307,-0.863232,-0.402780,0.442663,-1.005286,-0.952172,-0.666790
1250,0.265082,0.343986,0.256417,0.825024,0.383807,0.508198,-0.459309,0.964791,2.888387,-0.367494,...,0.344280,-0.182041,-0.204320,-0.476270,0.989903,1.593197,1.698665,-0.132963,-0.196666,1.225940
1251,-0.317913,0.069520,1.262713,1.208824,-0.239229,-0.141644,-0.449653,-0.603892,-0.708639,-0.323937,...,0.915209,-0.324163,1.140097,-0.040108,-1.376312,-1.116303,0.679353,-0.206692,-0.149487,-0.206514


In [543]:
tmp = pd.DataFrame(y_train)
tmp.value_counts()

0 
-1    1170
 1      83
Name: count, dtype: int64

In [544]:
print(X_train.columns[(X_train.mean() == 0) & (X_train.max() == 0) & (X_train.min() == 0)])

Index([18, 20, 84], dtype='int64')


In [545]:
X_train.mean()

0     -6.762348e-15
1     -2.010275e-15
2     -1.519756e-15
3      8.364330e-17
4     -1.000884e-14
           ...     
342   -3.969512e-17
343    1.842988e-17
344    1.984756e-16
345    1.913872e-16
346    8.321799e-16
Length: 347, dtype: float64

In [546]:
X_train.shape

(1253, 347)

In [547]:
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

'''lr = LogisticRegression(random_state=1234)  # 규제 강도 조절
rfe = RFE(lr, n_features_to_select=300)  # 특성 수를 늘림
X_train_rfe = rfe.fit_transform(X_train, y_train)
X_test_rfe = rfe.transform(X_test)


# PCA 적용
pca = PCA(n_components=100)
X_train = pca.fit_transform(X_train_rfe)
X_test = pca.transform(X_test_rfe)
'''

'lr = LogisticRegression(random_state=1234)  # 규제 강도 조절\nrfe = RFE(lr, n_features_to_select=300)  # 특성 수를 늘림\nX_train_rfe = rfe.fit_transform(X_train, y_train)\nX_test_rfe = rfe.transform(X_test)\n\n\n# PCA 적용\npca = PCA(n_components=100)\nX_train = pca.fit_transform(X_train_rfe)\nX_test = pca.transform(X_test_rfe)\n'

In [548]:
y_train = np.where(y_train == -1, 0, 1)
y_test = np.where(y_test == -1, 0, 1)

In [549]:
tmp = pd.DataFrame(y_train)
tmp.value_counts()

0
0    1170
1      83
Name: count, dtype: int64

In [550]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Convert the classes parameter to a numpy array

y_train = pd.DataFrame(y_train)


model = xgb.XGBClassifier(
    max_depth=5,
    learning_rate=0.05,
    min_child_weight=1,
    n_estimators=100,
    subsample=0.8,
    colsample_bytree=0.8,

)

model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [551]:

'''importances = model.feature_importances_

# 상위 200개의 중요한 특성 선택
indices = np.argsort(importances)[-200:]

# 특성 선택 시 iloc 사용
X_train = X_train.iloc[:, indices]
X_test = X_test.iloc[:, indices]'''



'importances = model.feature_importances_\n\n# 상위 200개의 중요한 특성 선택\nindices = np.argsort(importances)[-200:]\n\n# 특성 선택 시 iloc 사용\nX_train = X_train.iloc[:, indices]\nX_test = X_test.iloc[:, indices]'

In [552]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, cv=5, scoring='f1')
print(f'Cross-validated F1 scores: {scores}')


Cross-validated F1 scores: [0. 0. 0. 0. 0.]


In [553]:

scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])

model = xgb.XGBClassifier(
    max_depth=5,
    learning_rate=0.05,
    min_child_weight=1,
    n_estimators=100,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=1234,
    scale_pos_weight=scale_pos_weight
)

In [554]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, balanced_accuracy_score

param_grid = {
    'max_depth': [3, 6],
    'learning_rate': [0.01, 0.05],
    'n_estimators': [20, 50],
    'min_child_weight': [1, 2, 3],
}

grid_search = GridSearchCV(
    model,
    param_grid,
    scoring='f1',
    cv=2,
    verbose=2
)

grid_search.fit(X_train, y_train)

Fitting 2 folds for each of 24 candidates, totalling 48 fits
[CV] END learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=20; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=20; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=50; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=50; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, min_child_weight=2, n_estimators=20; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=3, min_child_weight=2, n_estimators=20; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=3, min_child_weight=2, n_estimators=50; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, min_child_weight=2, n_estimators=50; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, min_child_weight=3, n_estimators=20; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=3, min_child_weigh

GridSearchCV(cv=2,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=0.05, m...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=5,
                                     max_leaves=None, min_child_weight=1,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=100,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=1234, ...),
             param_grid={'learning_rate': [0.01, 0.05], 'max_depth': [3, 6],
                         'min_child_weight': [1, 2, 3],
                         'n_estimators': [20, 50]},
             scoring='f1', verbose=2)

In [555]:
# 최적의 모델 및 하이퍼파라미터 확인
model = grid_search.best_estimator_
print(f'Best model: {model}')
print(f'Best parameters: {grid_search.best_params_}')

# 테스트 데이터로 최적 모델 평가
model.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_test)[:, 1]
threshold = 0.1  # 임계값 조정
y_pred = (y_pred_proba >= threshold).astype(int)

f1 = f1_score(y_test, y_pred)
print(f'F1 score on the test set with best model: {f1}')

Best model: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=20, n_jobs=None,
              num_parallel_tree=None, random_state=1234, ...)
Best parameters: {'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 20}
F1 score on the test set with best model: 0.1253731343283582


In [556]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       293
           1       0.07      1.00      0.13        21

    accuracy                           0.07       314
   macro avg       0.03      0.50      0.06       314
weighted avg       0.00      0.07      0.01       314

[[  0 293]
 [  0  21]]


/opt/anaconda3/envs/forml/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/forml/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/forml/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [557]:
import joblib

joblib.dump(model, 'best_xgboost_model.pkl')

['best_xgboost_model.pkl']

In [558]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_pred, y_test)

array([[  0,   0],
       [293,  21]])